In [44]:
type FileEntry = 
    { 
        Name : string
        Size : int 
        }

type DirectoryEntry = 
    { 
        Name : string
        Parent : DirectoryEntry option
        Files : ResizeArray<FileEntry>
        Subdirectories :  ResizeArray<DirectoryEntry>
    }

module FileSystem =
    let getEmpty() = { Name = "/"; Parent = None; Files = ResizeArray(); Subdirectories = ResizeArray() }
    
    let addFile (d : DirectoryEntry) f = 
        d.Files.Add f
    
    let addSubdirectory (d: DirectoryEntry) name =
        let sd = { Name = name; Parent = Some d; Files = ResizeArray(); Subdirectories = ResizeArray() }
        d.Subdirectories.Add sd
        sd

    let rec findRoot (d : DirectoryEntry) =
        match d.Parent with
        | None -> d
        | Some p -> findRoot p

    let rec treeSize (d : DirectoryEntry) =
        let filesSize = 
            d.Files 
            |> Seq.sumBy (fun f -> f.Size)

        let subdirSizes = 
            d.Subdirectories 
            |> List.ofSeq
            |> List.map treeSize

        let subDirSize = subdirSizes |> List.sum
        filesSize + subDirSize

    let rec allFiles (d : DirectoryEntry) =
        Seq.append d.Files (d.Subdirectories |> Seq.collect allFiles)

    let rec allDirectories (d : DirectoryEntry) =
        d :: (d.Subdirectories |> Seq.collect allDirectories |> List.ofSeq)


In [45]:
let nextLineAndRest (text : string) = 
    let splitText = text.Split([|'\r';'\n'|], 2, StringSplitOptions.RemoveEmptyEntries)
    if splitText.Length <= 1 then
        text, ""
    else
        splitText[0], splitText[1]

let rec processListing (cwd : DirectoryEntry) (text : string) =
    if text <> "" then
        let thisLine, rest = nextLineAndRest text
        if thisLine.StartsWith "$" then
            processPrompt cwd text
        elif thisLine.StartsWith "dir" then
            // we'll add subdirectories when we enter them
            processListing cwd rest
        else
            let fileBits = thisLine.Split ' '
            FileSystem.addFile cwd { Name = fileBits[1]; Size = int fileBits[0] }
            processListing cwd rest

and processPrompt (cwd : DirectoryEntry) (text : string) =
    if text <> "" then
        let thisLine, rest = nextLineAndRest text
        
        if thisLine.StartsWith "$ cd" then
            let name = (thisLine.Split ' ')[2]
            if name = ".." then
                match cwd.Parent with
                | None -> failwith "Cannot go up from root"
                | Some p -> processPrompt p rest
            elif name = "/" then
                let d = FileSystem.findRoot cwd
                processPrompt d rest
            else
                let sd = FileSystem.addSubdirectory cwd name
                processPrompt sd rest
        elif thisLine.StartsWith "$ ls" then
            processListing cwd rest
        else
            failwith $"Expected a command, got '{thisLine}'"


In [ ]:
let allDirectorySizes root =
    FileSystem.allDirectories root
    |> List.map (
        fun d ->
            d.Name, FileSystem.treeSize d
    )

In [52]:
#r "nuget: FsUnit"
open FsUnitTyped

let testInput = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
"""

let root = FileSystem.getEmpty()
processPrompt root testInput

let testSizes = 
    allDirectorySizes root 
    |> Map.ofList

testSizes["e"] |> shouldEqual 584
testSizes["a"] |> shouldEqual 94853 
testSizes["d"] |> shouldEqual 24933642
testSizes["/"] |> shouldEqual 48381165

let totalSmall =
    testSizes
    |> Map.toSeq
    |> Seq.filter (fun (_,s) -> s < 100000)
    |> Seq.sumBy snd

totalSmall |> shouldEqual 95437


Installed Packages FsUnit, 5.1.0

In [ ]:
open System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_07.txt")
let listing = File.ReadAllText(sourcePath)

let root = FileSystem.getEmpty()
processPrompt root listing